In [1]:
######## Importing the necessary libraries #########


import pandas as pd
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
%matplotlib inline   

In [2]:
##### Fetching the cleaned FM transmitter data for the countries: CANADA, AUSTRALIA and GERMANY ####
#####  EIRP in Watts #####
##### Latitude ranges from 90(N.Pole) to 0 (EQUATOR) to -90(S.Pole) ########
##### Longitude ranges between 0 and 360 degree eastwards ############


#Reading the CSV file
df=pd.read_csv("/home/ghoshsonia/rfi/Final_Revised.csv")

#Removing the Null/missing values in the CSV file
df.dropna(subset = ["Latitude in degrees"], inplace=True)


#df.set_index("Area Served", inplace = True) # Set 'Area Served' column as index on a Dataframe
#df=df.loc[["Perth", "Perth City"]] # Using the operator .loc[] to select multiple rows from the Dataframe
df

,Latitude in degrees,Longitude in degrees,Frequency(MHz),EIRP
0,50.947181,8.532986,87.6,164000.0
1,53.509289,10.093932,87.6,131200.0
2,49.832397,9.592726,87.6,41000.0
3,49.629033,7.820645,87.6,820.0
4,49.681928,7.327415,87.6,328.0
...,...,...,...,...
14102,-25.234722,130.991944,99.7,524.8
14103,-25.234722,130.991944,100.5,524.8
14104,-25.234722,130.991944,102.1,328.0
14105,-25.234722,130.991944,105.3,656.0


In [3]:
np.shape(df)

(13195, 4)

In [4]:
###### User defined frequency axis ######################
fstart=87
fstop=108
fres=0.5
freq=np.arange(fstart,fstop,fres)
data_freq=df['Frequency(MHz)'].values
diff=np.zeros((len(data_freq),len(freq)),dtype=object)
freq_arr=np.zeros(len(data_freq))


for i in range(len(data_freq)):
    for j in range(len(freq)):
        diff[i][j]=abs(data_freq[i]-freq[j])
        freq_arr[i]=freq[np.argmin(diff[i])]

In [5]:
df['New Frequency']=freq_arr
df

,Latitude in degrees,Longitude in degrees,Frequency(MHz),EIRP,New Frequency
0,50.947181,8.532986,87.6,164000.0,87.5
1,53.509289,10.093932,87.6,131200.0,87.5
2,49.832397,9.592726,87.6,41000.0,87.5
3,49.629033,7.820645,87.6,820.0,87.5
4,49.681928,7.327415,87.6,328.0,87.5
...,...,...,...,...,...
14102,-25.234722,130.991944,99.7,524.8,99.5
14103,-25.234722,130.991944,100.5,524.8,100.5
14104,-25.234722,130.991944,102.1,328.0,102.0
14105,-25.234722,130.991944,105.3,656.0,105.5


In [6]:
#Resolution of the map
print('-----------Resolution of the map---------') 
nside = 32
print("The number of pixels for the given NSIDE: " + str(hp.nside2npix(nside)))
print("Approximate resolution in degrees for given nside: " + str(np.degrees(hp.nside2resol(nside))))
print( "Pixel area: %.2f square degrees" % hp.nside2pixarea(nside, degrees=True))



-----------Resolution of the map---------
The number of pixels for the given NSIDE: 12288
Approximate resolution in degrees for given nside: 1.8322594196359498
Pixel area: 3.36 square degrees


In [7]:

######-------------------Allocating Pixel number to the Latitude and Longitude of each Tx in the CSV---------#######



# Healpy pixel number when input angles are assumed to be longitude and latitude in degrees
pixel_indices = hp.ang2pix(nside, df['Longitude in degrees'].to_numpy() ,df['Latitude in degrees'].to_numpy(),lonlat=True)
df['Pixel_number']=pixel_indices 
print(pixel_indices)

df

[1302 1106 1406 ... 8686 8686 8821]


,Latitude in degrees,Longitude in degrees,Frequency(MHz),EIRP,New Frequency,Pixel_number
0,50.947181,8.532986,87.6,164000.0,87.5,1302
1,53.509289,10.093932,87.6,131200.0,87.5,1106
2,49.832397,9.592726,87.6,41000.0,87.5,1406
3,49.629033,7.820645,87.6,820.0,87.5,1406
4,49.681928,7.327415,87.6,328.0,87.5,1406
...,...,...,...,...,...,...
14102,-25.234722,130.991944,99.7,524.8,99.5,8686
14103,-25.234722,130.991944,100.5,524.8,100.5,8686
14104,-25.234722,130.991944,102.1,328.0,102.0,8686
14105,-25.234722,130.991944,105.3,656.0,105.5,8686


In [8]:
#######---------------Conversion of the pixel numbers w.r.t the given NSIDE to corresponding angular coordinates--------#######


NPIX = hp.nside2npix(nside) # Storing the number of pixels of the map corresponding to the given NSIDE
arr=np.arange(NPIX) #Create an an array of pixel numbers with respect to the NSIDE

theta, phi = (hp.pix2ang(nside, ipix=arr,lonlat=True)) # Array of the angular coordinates co-latitude(theta) 
                                                        #and longitude(phi) in degrees 
                                                       # With respect to the given NSIDE


In [9]:
#######------------------Altitudes of the satellite orbit--------####


#Creating user defined data points between 400 km to 36000 km in log scale
data_point=int(input("Enter number of data points : "))
altitude= np.logspace(np.log10(400),np.log10(36000),data_point) 
print(altitude)

Enter number of data points : 3
[  400.         3794.7331922 36000.       ]


In [10]:
###########----------------- Calculation of Field of view of the satellite---------#######
###########-----------------Considering Nadir-pointing Field of View Geometry-------######
##########-----------------Considering the FOV of the satellite to be tangent to the surface of the Earth------######


FOV=np.zeros(len(altitude))
for i in range(0,len(altitude)):
# Consider a case of full coverage under elevation of 0 º
    Rad= 6371 # Mean radius of Earth in km
    FOV[i]= 2*np.arcsin(Rad/(Rad+ altitude[i]))  # Field of view for maximal coverage in radians when elevation is 0 º 
    print(" The Field of view of the satellite at a height of",altitude[i],"km is",FOV[i], "radians")

 The Field of view of the satellite at a height of 400.0000000000001 km is 2.4507006996152447 radians
 The Field of view of the satellite at a height of 3794.733192202054 km is 1.3546564445594882 radians
 The Field of view of the satellite at a height of 35999.99999999996 km is 0.30186940950576446 radians


In [11]:
###########----------------- Calculation of the radius of the FOV of the satellite---------#######



# The surface of the coverage area of the Earth depends on the central angle
Central_angle=np.zeros(len(altitude))
for i in range(0,len(altitude)):
    Central_angle[i]=np.arccos(Rad/(Rad+altitude[i])) # Central angle in radians
    
    Dia_of_FOV=2*Central_angle*Rad  # Diameter of the FOV (disc on the Earth's surface)in km
    Rad_of_FOV= Dia_of_FOV/2 # Radius of the FOV (disc on the Earth's surface)in km
    Rad_of_FOV=Rad_of_FOV/Rad  # Radius of the FOV in Radians
    print(" The Radius of the Field of View for a height of",altitude[i],"km in radians is",Rad_of_FOV[i])
len(Rad_of_FOV)

 The Radius of the Field of View for a height of 400.0000000000001 km in radians is 0.3454459769872743
 The Radius of the Field of View for a height of 3794.733192202054 km in radians is 0.8934681045151526
 The Radius of the Field of View for a height of 35999.99999999996 km in radians is 1.4198616220420144


3

In [12]:
###############-----Storing indices of the pixel number that are inside the circle/disc(FOV) wrt the altitude-----------######


vec1 = hp.ang2vec(theta,phi,lonlat=True) #Using ang2vec convert angles that is co-latitude and longitude in radians
                                        #to 3D position vector
    
disc=np.zeros((len(arr),len(Rad_of_FOV)),dtype=object)#Array of indices of the pixel number that are inside the 
                                                      #circle/disc specified by vec and radius
def pixel_disc(a,b):
    for i in range(len(a)):
        for j in range(len(b)):
            disc[j][i]=hp.query_disc(nside, vec1[j], radius=a[i])
pixel_disc(Rad_of_FOV,arr)

In [13]:
df1 = pd.DataFrame(disc)
df1

,0,1,2
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
2,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
3,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
4,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
...,...,...,...
12283,"[11808, 11809, 11858, 11859, 11860, 11861, 118...","[9792, 9793, 9794, 9795, 9796, 9797, 9798, 979...","[6720, 6832, 6833, 6834, 6835, 6836, 6837, 683..."
12284,"[11868, 11869, 11870, 11871, 11872, 11873, 118...","[9920, 9921, 9922, 9923, 9924, 9925, 9926, 992...","[6976, 6977, 6978, 6979, 6980, 6981, 6982, 698..."
12285,"[11880, 11881, 11882, 11883, 11884, 11885, 118...","[9938, 9939, 9940, 9941, 9942, 9943, 9944, 994...","[6996, 6997, 6998, 6999, 7000, 7001, 7002, 700..."
12286,"[11894, 11895, 11896, 11897, 11898, 11899, 119...","[9970, 9971, 9972, 9973, 9974, 9975, 9976, 997...","[7028, 7029, 7030, 7031, 7032, 7033, 7034, 703..."


In [14]:
######-----Storing indices of the pixel number that are common between the FOV disc and the satellite pixel no------####


Comm_pix=np.zeros((len(disc[:,i]),len(Rad_of_FOV)),dtype=object)

for i in range(len(Rad_of_FOV)):
    for k in range(len(disc[:,i])):
     

         Comm_pix[k][i]=np.intersect1d(pixel_indices,disc[k][i])#Array of indices of the pixel number that are
                                                                #common between the FOV disc and the satellite pixel no
         

In [15]:
df2 = pd.DataFrame(Comm_pix)
df2

,0,1,2
0,"[209, 253, 301]","[209, 253, 301, 352, 399, 400, 401, 403, 405, ...","[209, 253, 301, 352, 399, 400, 401, 403, 405, ..."
1,"[209, 253, 301]","[209, 253, 301, 352, 399, 400, 401, 403, 405, ...","[209, 253, 301, 352, 399, 400, 401, 403, 405, ..."
2,"[209, 253, 301, 352, 399, 400, 401, 403, 405, ...","[209, 253, 301, 352, 399, 400, 401, 403, 405, ...","[209, 253, 301, 352, 399, 400, 401, 403, 405, ..."
3,"[209, 253, 301, 352, 405, 407, 409]","[209, 253, 301, 352, 399, 400, 401, 403, 405, ...","[209, 253, 301, 352, 399, 400, 401, 403, 405, ..."
4,"[209, 253, 301]","[209, 253, 301, 352, 399, 400, 401, 403, 405, ...","[209, 253, 301, 352, 399, 400, 401, 403, 405, ..."
...,...,...,...
12283,[],"[10227, 10228, 10354]","[6947, 7394, 7406, 7407, 7408, 7410, 7533, 753..."
12284,[],"[10099, 10100, 10227, 10228, 10354]","[7141, 7154, 7155, 7279, 7283, 7394, 7406, 740..."
12285,[],"[9970, 9971, 9972, 10099, 10100, 10227, 10228,...","[7141, 7154, 7155, 7279, 7283, 7394, 7406, 740..."
12286,[],"[9970, 9971, 9972, 10099, 10100, 10227, 10228,...","[7141, 7154, 7155, 7279, 7283, 7394, 7406, 740..."


In [17]:
######-----Storing indices of the pixel number that are common between the FOV disc and the satellite pixel no ------#


Comm_TX=np.zeros((len(Comm_pix[:,j]),len(Rad_of_FOV)),dtype=object)
found_common=np.zeros((len(Comm_pix[:,j]),len(Rad_of_FOV)),dtype=object)

for j in range(len(Rad_of_FOV)):
    for i in range(len(Comm_pix[:,j])):
        Comm_TX[i][j]=set(Comm_pix[i][j])
        found_common[i][j] = [l for l in pixel_indices if l in Comm_TX[i][j]]#Array of indices of the pixel number that 
                                                    #are common between the FOV disc and the satellite pixel no with
                                                    # Tx having same pixel number
                                                                       
         

In [18]:
df3 = pd.DataFrame(found_common)
df3

,0,1,2
0,"[253, 301, 209]","[1302, 1106, 1406, 1406, 1406, 1300, 1406, 140...","[1302, 1106, 1406, 1406, 1406, 1300, 1406, 140..."
1,"[253, 301, 209]","[1302, 1106, 1406, 1406, 1406, 1300, 1406, 140...","[1302, 1106, 1406, 1406, 1406, 1300, 1406, 140..."
2,"[401, 400, 399, 253, 403, 403, 403, 407, 301, ...","[1302, 1106, 1406, 1406, 1406, 1300, 1406, 140...","[1302, 1106, 1406, 1406, 1406, 1300, 1406, 140..."
3,"[253, 409, 407, 301, 209, 405, 405, 352, 405, ...","[1302, 1106, 1406, 1406, 1406, 1300, 1406, 140...","[1302, 1106, 1406, 1406, 1406, 1300, 1406, 140..."
4,"[253, 301, 209]","[1302, 1106, 1406, 1406, 1406, 1300, 1406, 140...","[1302, 1106, 1406, 1406, 1406, 1300, 1406, 140..."
...,...,...,...
12283,[],"[10228, 10228, 10228, 10354, 10354, 10354, 103...","[9971, 6947, 9971, 9713, 9713, 9713, 9713, 971..."
12284,[],"[10100, 10100, 10100, 10228, 10228, 10228, 101...","[9971, 9971, 9713, 9713, 9713, 9713, 9713, 971..."
12285,[],"[9971, 9971, 9971, 9971, 9971, 9971, 9971, 997...","[9971, 9971, 9713, 9713, 9713, 9713, 9713, 971..."
12286,[],"[9971, 9971, 9971, 9971, 9971, 9971, 9971, 997...","[9971, 9971, 9713, 9713, 9713, 9713, 9713, 971..."


In [19]:
#######-------Store the values of the received power in Watt,dBm and Kelvin wrt altitude----#####
######------- Calculation of the received power using Friis Transmission Equation--------#######
######-------Considering isotropic transmitter and receiver with gain =1 -------------#########





res=fres*1e6 #Bandwidth
Rx_Power= np.zeros((len(df),len(altitude)))
Rx_Power_in_Kelvin=np.zeros((len(df),len(altitude)))
for i in range(0,len(altitude)):
    for j in range(0,len(df)):
        wavelength= (3*1e8)/(df.iloc[j]['Frequency(MHz)']*1e6)
        Rx_Power[j][i]= ((df.iloc[j]['EIRP'])*(wavelength)**2)/(4*np.pi*altitude[i])**2 #the Friis Transmission Equation
        
        Rx_Power_in_Kelvin[j][i]=Rx_Power[j][i]/(1.38*1e-23*res)#in Kelvin

In [29]:
df9=pd.DataFrame(Rx_Power_in_Kelvin)
df9['Pixel_number']=pixel_indices
df9['New Frequency']=freq_arr
#df9=df9.groupby(['New Frequency']).sum()

df9

,0,1,2,Pixel_number,New Frequency
0,1.103289e+16,1.225877e+14,1.362085e+12,1302,87.5
1,8.826313e+15,9.807015e+13,1.089668e+12,1106,87.5
2,2.758223e+15,3.064692e+13,3.405213e+11,1406,87.5
3,5.516446e+13,6.129384e+11,6.810427e+09,1406,87.5
4,2.206578e+13,2.451754e+11,2.724171e+09,1406,87.5
...,...,...,...,...,...
13190,2.725569e+13,3.028410e+11,3.364900e+09,8686,99.5
13191,2.682350e+13,2.980389e+11,3.311543e+09,8686,100.5
13192,1.624337e+13,1.804819e+11,2.005354e+09,8686,102.0
13193,3.054223e+13,3.393582e+11,3.770646e+09,8686,105.5


In [ ]:
df8=pd.DataFrame(Rx_Power_in_Kelvin)

In [ ]:
power_output=np.zeros((len(Rad_of_FOV),nside,len(freq_arr)),dtype=object)

g=np.zeros(len(Rad_of_FOV),len(found_common))
for j in range(len(Rad_of_FOV)):
    for k in range(nside):
        for l in range(len(freq_arr)):
             if df9['Pixel_number'].isin(found_common[k][j]):
                    df11=df9